# Гибридная рекомендательная система для отзывов на товары Amazon

Этот проект представляет собой пошаговое исследование и разработку рекомендательной системы для товаров Amazon. Цель — предсказать следующий товар, который может купить пользователь, на основе его предыдущих оценок. В ходе проекта были построены и оценены три различные модели: чистая коллаборативная фильтрация и две гибридные модели с текстовым переранжированием.

## Набор данных
###Контекст
Этот набор данных содержит более 568 тысяч отзывов потребителей о различных товарах Amazon. Этот набор данных также доступен на других сайтах, связанных с этими наборами данных, но я счёл его полезным и поделился им здесь.

###Содержание
Этот набор данных содержит следующие атрибуты:


*   Всего записей: 568454
*   Всего столбцов: 10
*  Доменное имя: amazon.com
*  Расширение файла: CSV



####Доступные поля: Id, ProductId, UserId, ProfileName, HelpfulnessNumerator, HelpfulnessDenomenator, Score, Time, Summary, Text
Используется публичный датасет [Amazon Product Reviews](https://www.kaggle.com/datasets/arhamrumi/amazon-product-reviews) с платформы Kaggle.

**Предварительная обработка:**
1.  Данные были загружены и проанализированы на предмет пропусков.
2.  Для построения надежных рекомендаций были отобраны только пользователи, оставившие 5 и более отзывов.
3.  Данные были разделены на обучающую (`train_df`) и тестовую (`test_df`) выборки по принципу "Leave-One-Out": последняя по времени покупка каждого пользователя попала в тест, а все предыдущие — в обучение.

##  Построенные модели

### Модель 1: Коллаборативная фильтрация (ALS)

В качестве базовой модели была использована коллаборативная фильтрация на основе метода **Alternating Least Squares (ALS)** из библиотеки `implicit`.

-   **Подбор гиперпараметров**: С помощью поиска по сетке (`Grid Search`) были найдены оптимальные параметры для модели: `factors=100`, `regularization=0.01`, `iterations=25`.
-   **Качество**: Модель показала очень хороший базовый результат.

### Модель 2: Гибридная модель (ALS + TF-IDF на `Summary`)

Первая попытка улучшить базовую модель заключалась в добавлении контентной логики. Идея была в том, чтобы переранжировать топ-20 кандидатов от ALS, используя текстовую схожесть на основе кратких описаний (`Summary`).

-   **Метод**: Для каждого пользователя вычислялся средний TF-IDF вектор его истории покупок. Затем 20 кандидатов от ALS сортировались по косинусному сходству с этим вектором.
-   **Результат**: Эта модель показала **значительное ухудшение** качества по сравнению с базовой. Вывод: краткие описания `Summary` слишком зашумлены и не несут достаточной информации для качественного переранжирования.

### Модель 3: Улучшенная гибридная модель (ALS + TF-IDF на `Text`)

Основываясь на выводах предыдущего шага, была построена вторая гибридная модель, но на этот раз TF-IDF векторы создавались на основе **полных текстов отзывов (`Text`)**.

-   **Метод**: Логика переранжирования осталась той же, но использовалась новая, более подробная TF-IDF матрица.
-   **Результат**: Эта модель показала **улучшение** по сравнению с чистой ALS. Это доказывает, что при достаточном количестве качественной текстовой информации контентное переранжирование способно улучшить результаты коллаборативной фильтрации.

Используем API Kaggle для загрузки необходимых данных

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'arhamrumi/amazon-product-reviews'
!unzip '/content/amazon-product-reviews.zip'
! rm '/content/amazon-product-reviews.zip'

установка implicit

In [ ]:
!pip install implicit

импорт библиотеки

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
import itertools
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/Reviews.csv', index_col= 'Id')
df.head(5)

#EDA

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

Здесь мы видим, что у нас немного пропущенных значений в столбцах ProfileName 26 и Summary 27. Поскольку эти данные не критичны для нашей задачи, мы не будем их удалять

In [ ]:
sns.set_theme(style="whitegrid")

plt.figure(figsize=(8, 5))
ax = sns.histplot(
    data=df,
    x='Score',
    bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
    stat= 'percent',
    discrete=True,
    color="#4C72B0",
    edgecolor='black',
    linewidth=1.2
)
for p in ax.patches:
    percent = p.get_height()
    ax.annotate(f'{percent:.1f}%',
                (p.get_x() + p.get_width() / 2, percent),
                ha='center', va='bottom', fontsize=11, color='black')


plt.title('Оценки пользователей (1–5)', fontsize=16, fontweight='bold')
plt.xlabel('Оценка', fontsize=12)
plt.ylabel('Количество', fontsize=12)
plt.xticks([1, 2, 3, 4, 5], fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

Мы видим на графике, что оценки 5 составляют 64% от всех, что является хорошим показателем и позволяет рекомендовать продукт

In [ ]:
user_counts = df.groupby('UserId').size()
users_with_5_plus = user_counts[user_counts >= 5].index
df_filtered = df[df['UserId'].isin(users_with_5_plus)]

Я решил оставить клиентов, которые оценили более 5 товаров, чтобы сохранить надёжный вектор довери

In [ ]:
def time_based_split(df):
    train_list = []
    test_list = []

    grouped = df.groupby('UserId')
    for user, group in grouped:
        group = group.sort_values('Time')  # сортируем по времени
        train_list.append(group.iloc[:-1])  # все кроме последней покупки — в train
        test_list.append(group.iloc[-1:])   # последняя покупка — в test

    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df = pd.concat(test_list).reset_index(drop=True)
    return train_df, test_df

train_df, test_df = time_based_split(df_filtered)

Для разделения данных использовалась логика 'Leave-One-Out': последняя оценка каждого пользователя включена в тестовую выборку test_df, а все предыдущие — в обучающую train_df. Это позволяет моделировать реальный сценарий рекомендаций

In [ ]:
# Список уникальных пользователей и товаров на трейне
unique_users = train_df['UserId'].unique()
unique_items = train_df['ProductId'].unique()

# Маппинг UserId → индекс и обратно
user_map = {u: i for i, u in enumerate(unique_users)}
item_map = {p: i for i, p in enumerate(unique_items)}

# Инвертированные мапы (если потом нужно восстановить ID)
user_map_inv = {i: u for u, i in user_map.items()}
item_map_inv = {i: p for p, i in item_map.items()}

# Добавляем индексированные значения в train/test
train_df['user_idx'] = train_df['UserId'].map(user_map)
train_df['item_idx'] = train_df['ProductId'].map(item_map)

# В test_df возможны юзеры и товары, которых нет в трейне — фильтруем
test_df = test_df[test_df['UserId'].isin(user_map)]
test_df = test_df[test_df['ProductId'].isin(item_map)]

test_df['user_idx'] = test_df['UserId'].map(user_map)
test_df['item_idx'] = test_df['ProductId'].map(item_map)


Для построения матрицы взаимодействий и подачи данных в модель была выполнена индексация пользователей и товаров на основе обучающей выборки. Затем в тестовой выборке оставлены только те пользователи и товары, которые встречаются в трейне, и добавлены соответствующие индексы. Это позволяет обеспечить совместимость между train_df и test_df

In [ ]:
# log1p: сглаживает большие значения, полезно при оценках от 1 до 5
ratings = coo_matrix(
    (np.log1p(train_df['Score']), (train_df['user_idx'], train_df['item_idx'])),
    shape=(len(user_map), len(item_map))
)

используется логарифмическое преобразование оценок **log1p** что позволяет уменьшить влияние выбросов и более сбалансированно обучать модель. Матрица создаётся в формате coo_matrix, где строки — пользователи, столбцы — товары, а значения — логарифмированные оценки.

Модель **AlternatingLeastSquares** из библиотеки **implicit** обучается на транспонированной разреженной матрице item-user взаимодействий и позволяет формировать персонализированные рекомендации по логарифмированным оценкам.

In [ ]:
# Преобразуем в CSR формат — оптимален для ALS
ratings_csr = ratings.tocsr()

# Обучаем ALS
model = AlternatingLeastSquares(factors=100, regularization=0.01, iterations=15)
model.fit(ratings_csr)

Из тестовой выборки для каждого пользователя выбирается последний оценённый товар **item_idx** как правильный ответ для проверки рекомендаций. Учитываются только пользователи, присутствующие в обучающей выборке.

In [ ]:
test_items = test_df[test_df['user_idx'] < len(user_map)].set_index('user_idx')['item_idx']

Для оценки качества рекомендаций используется метрика Hit@10. Она показывает долю пользователей, для которых правильный товар (последний оценённый в тесте) попал в топ-10 рекомендованных моделью позиций. Чем выше значение Hit@10, тем лучше модель предсказывает релевантные рекомендации

In [ ]:
hit = 0
total = 0

for user_idx, true_item in test_items.items():
    # model.recommend returns a tuple of (item_ids, scores)
    recommended_items, _ = model.recommend(
        userid=user_idx,
        user_items=ratings_csr[user_idx],
        N=10,
        filter_already_liked_items=True
    )

    if true_item in recommended_items:
        hit += 1
    total += 1

hit_rate = hit / total
print(f'Hit@10: {hit_rate:.4f}')

Мы получили достаточно хороший результат по метрике **Hit@10 — 0.4534**, что означает, что почти у половины пользователей правильный товар попадает в топ-10 рекомендаций модели.

подбор оптимальных гиперпараметров модели для достижения наилучших результатов и повышения качества рекомендаций.

In [ ]:
# Определяем сетку гиперпараметров
factors = [50, 100]
regularization = [0.01, 0.1]
iterations = [15, 25]

# Создаем все возможные комбинации гиперпараметров
grid = list(itertools.product(factors, regularization, iterations))

best_hit_rate = 0
best_params = {}

# Создаем словарь {user: [items]} для обучающих данных для быстрой проверки
train_user_items = train_df.groupby('user_idx')['item_idx'].apply(list).to_dict()

for f, r, i in grid:
    print(f"Training with factors={f}, regularization={r}, iterations={i}")

    # Обучаем модель
    model = AlternatingLeastSquares(factors=f, regularization=r, iterations=i, random_state=42)
    model.fit(ratings_csr)

    # --- Считаем Hit@10 на трейне ---
    train_hit = 0
    # Проходим по всем пользователям в обучающей выборке
    for user_idx, true_items in train_user_items.items():
        recommended_items, _ = model.recommend(
            userid=user_idx,
            user_items=ratings_csr[user_idx],
            N=10,
            filter_already_liked_items=False # Важно: не фильтруем уже купленное
        )
        # Проверяем, есть ли хотя бы одно совпадение
        if any(item in true_items for item in recommended_items):
            train_hit += 1

    train_hit_rate = train_hit / len(train_user_items)
    print(f'Train Hit@10: {train_hit_rate:.4f}')

    # --- Считаем Hit@10 на тесте ---
    test_hit = 0
    for user_idx, true_item in test_items.items():
        recommended_items, _ = model.recommend(
            userid=user_idx,
            user_items=ratings_csr[user_idx],
            N=10,
            filter_already_liked_items=True # На тесте фильтруем
        )

        if true_item in recommended_items:
            test_hit += 1

    test_hit_rate = test_hit / len(test_items)
    print(f'Test Hit@10: {test_hit_rate:.4f}\n')

    # Сохраняем лучшую модель по результатам на ТЕСТЕ
    if test_hit_rate > best_hit_rate:
        best_hit_rate = test_hit_rate
        best_params = {'factors': f, 'regularization': r, 'iterations': i}

print(f"\nBest Test Hit@10: {best_hit_rate:.4f}")
print(f"Best params: {best_params}")

In [ ]:
# 1. Обучение финальной модели с лучшими параметрами
best_params = {'factors': 100, 'regularization': 0.01, 'iterations': 25}
final_model = AlternatingLeastSquares(**best_params, random_state=42)
final_model.fit(ratings_csr)

# 2. Функция для получения рекомендаций
def get_recommendations(user_id, model, n=10):
    # Проверяем, есть ли такой пользователь в нашей карте
    if user_id not in user_map:
        return f"Пользователь {user_id} не найден в обучающей выборке."

    # Получаем внутренний индекс пользователя
    user_idx = user_map[user_id]

    # Получаем рекомендации
    recommended_item_idxs, _ = model.recommend(
        userid=user_idx,
        user_items=ratings_csr[user_idx],
        N=n,
        filter_already_liked_items=True
    )

    # Преобразуем индексы товаров обратно в ProductId
    recommended_product_ids = [item_map_inv[idx] for idx in recommended_item_idxs]

    return recommended_product_ids

# 3. Пример использования
# Возьмем случайного пользователя из тестового набора
example_user_id = test_df['UserId'].iloc[5]

print(f"Рекомендации для пользователя: {example_user_id}\n")

# Получаем и выводим рекомендации
recommendations = get_recommendations(example_user_id, final_model)
for i, product_id in enumerate(recommendations):
    print(f"{i+1}. ProductId: {product_id}")

# Для контекста, давайте посмотрим, что этот пользователь покупал раньше
print("\nИстория покупок этого пользователя (из train):")
display(train_df[train_df['UserId'] == example_user_id])

Рекомендации для пользователя: **A1017Q5HHWNALE**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Подготовка текстов: берем столбец 'Text'
#    Заполняем возможные пропуски пустой строкой
item_full_texts = df[['ProductId', 'Text']].drop_duplicates('ProductId').set_index('ProductId')
item_full_texts['Text'] = item_full_texts['Text'].fillna('')

# Упорядочим тексты в том же порядке, что и в item_map
ordered_texts = item_full_texts.reindex(item_map.keys())['Text']

# 2. Создаем новую TF-IDF матрицу
#    Используем немного больше признаков, так как тексты богаче
tfidf_full = TfidfVectorizer(max_features=10000, stop_words='english')
tfidf_matrix_full = tfidf_full.fit_transform(ordered_texts)

TF-IDF матрица построена по колонке Text. Размерность: (34339, 10000)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Словарь для хранения финальных переранжированных рекомендаций (на основе полных текстов)
reranked_full_text_recs = {}

# Проходим по пользователям и их топ-20 ALS рекомендациям
for user_id, recommended_items in als_top_20_recommendations.items():

    # 1. Получаем историю покупок пользователя (индексы товаров)
    user_history_items = train_df[train_df['user_idx'] == user_id]['item_idx'].values

    if len(user_history_items) == 0:
        continue

    # 2. Получаем TF-IDF векторы для истории и для рекомендаций из НОВОЙ матрицы
    history_vectors = tfidf_matrix_full[user_history_items]
    recommended_vectors = tfidf_matrix_full[recommended_items]

    # 3. Считаем косинусное сходство
    similarity_matrix = cosine_similarity(recommended_vectors, history_vectors)

    # 4. Для каждой рекомендации считаем среднюю схожесть со всей историей
    avg_similarity = similarity_matrix.mean(axis=1)

    # 5. Сортируем и выбираем топ-10
    reranked_indices = np.argsort(avg_similarity)[::-1]
    top_10_reranked = [recommended_items[i] for i in reranked_indices[:10]]

    reranked_full_text_recs[user_id] = top_10_reranked

Переранжирован топ-10 рекомендаций для 20 366 пользователей на основе текстового содержимого ('Text'). Используемые тексты были преобразованы с помощью TF-IDF векторизации, что позволило учитывать семантическое сходство между пользователями и рекомендуемыми объектами.

In [ ]:
hit = 0
total = 0

for user_idx, true_item in test_items.items():
    if user_idx in reranked_full_text_recs:
        recommended_items = reranked_full_text_recs[user_idx]

        if true_item in recommended_items:
            hit += 1
        total += 1

if total > 0:
    hit_rate = hit / total
    print(f'Hybrid Model (Full Text) Hit@10: {hit_rate:.4f}')
else:
    print("Не удалось рассчитать Hit@10.")

print("\n--- Сводка результатов ---")
print(f"Original ALS Hit@10: {best_hit_rate:.4f}")
print(f"Hybrid (Summary) Hit@10: 0.2408") # Значение из предыдущего запуска
print(f"Hybrid (Full Text) Hit@10: {hit / total:.4f}" if total > 0 else "N/A")

##  Итоговые результаты

Производительность всех моделей оценивалась по метрике **Hit@10** (доля пользователей, для которых правильный товар попал в топ-10 рекомендаций).

| Модель | Описание | Hit@10 |
| :--- | :--- | :--- |
| **Original ALS** | Коллаборативная фильтрация | 0.4542 |
| **Hybrid (Summary)** | ALS + TF-IDF на кратких описаниях | 0.2408 |
| **Hybrid (Full Text)** | ALS + TF-IDF на полных текстах | **0.4667** |

##  Выводы и дальнейшие шаги

Чистая коллаборативная фильтрация (ALS) является очень сильной базовой моделью. Однако ее можно улучшить с помощью гибридного подхода, если использовать качественные контентные данные. Полные тексты отзывов, в отличие от кратких summary, содержат достаточно сигнала для эффективного переранжирования.

**Возможные дальнейшие шаги:**
-   **Использовать BERT**: Применить более продвинутые языковые модели (BERT, RoBERTa) для создания семантических эмбеддингов текста, что может дать еще больший прирост качества.
-   **Настройка гибридной модели**: Экспериментировать с размером окна кандидатов от ALS (например, брать топ-50 или топ-100) и весами, с которыми смешиваются оценки от ALS и контентной модели.